In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/30 17:52:30 WARN Utils: Your hostname, DESKTOP-CN01VS3 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/30 17:52:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/30 17:52:28 INFO SparkContext: Running Spark version 3.5.0
25/03/30 17:52:28 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/03/30 17:52:28 INFO SparkContext: Java version 17.0.14
25/03/30 17:52:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/30 17:52:28 INFO ResourceUtils: ==============================================================
25/03/30 17:52:28 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/30 17:52:28 INFO ResourceUtils: ==============================================================
25/03/30 17:52:28 INFO SparkCon

import $ivy.$
import org.apache.spark.sql.SparkSession
spark: SparkSession = org.apache.spark.sql.SparkSession@71dc2e6a
import spark.implicits._

In [2]:
import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.udaf

case class IntSumBuffer(var sum: Long = 0L)

object SumAll extends Aggregator[Int, IntSumBuffer, Long] {
  def zero = IntSumBuffer(0L)
  def reduce(buffer: IntSumBuffer, value: Int): IntSumBuffer = {
    buffer.sum += value
    buffer
  }
  def merge(b1: IntSumBuffer, b2: IntSumBuffer): IntSumBuffer = {
    b1.sum += b2.sum
    b1
  }
  def finish(reduction: IntSumBuffer): Long = reduction.sum
  def bufferEncoder = Encoders.product[IntSumBuffer]
  def outputEncoder = Encoders.scalaLong
}

val sumAll = udaf(SumAll)


import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.udaf
defined class IntSumBuffer
defined object SumAll
sumAll: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedAggregator(
  aggregator = ammonite.$sess.cmd2$Helper$SumAll$@10b31a79,
  inputEncoder = ExpressionEncoder(
    objSerializer = BoundReference(
      ordinal = 0,
      dataType = IntegerType,
      nullable = false
    ),
    objDeserializer = AssertNotNull(
      child = UpCast(
        child = GetColumnByOrdinal(ordinal = 0, dataType = IntegerType),
        target = IntegerType,
        walkedTypePath = List("- root class: \"int\"")
      ),
      walkedTypePath = List("- root class: \"int\"")
    ),
    clsTag = Int
  ),
  name = None,
  nullable = true,
  deterministic = true
)

In [3]:
val df = Seq(1, 2, 3, 4, 5).toDF("value")
df.select(sumAll($"value").as("total_sum")).show()

25/03/30 17:52:31 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/03/30 17:52:31 INFO SharedState: Warehouse path is 'file:/home/eyal/code/notebooks/spark-warehouse'.
25/03/30 17:52:32 INFO CodeGenerator: Code generated in 158.959275 ms
25/03/30 17:52:33 INFO DAGScheduler: Registering RDD 3 (show at cmd3.sc:2) as input to shuffle 0
25/03/30 17:52:33 INFO DAGScheduler: Got map stage job 0 (show at cmd3.sc:2) with 5 output partitions
25/03/30 17:52:33 INFO DAGScheduler: Final stage: ShuffleMapStage 0 (show at cmd3.sc:2)
25/03/30 17:52:33 INFO DAGScheduler: Parents of final stage: List()
25/03/30 17:52:33 INFO DAGScheduler: Missing parents: List()
25/03/30 17:52:33 INFO DAGScheduler: Submitting ShuffleMapStage 0 (MapPartitionsRDD[3] at show at cmd3.sc:2), which has no missing parents
25/03/30 17:52:33 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 21.3 KiB, free 2.2 GiB)
25/03/30 17:52:33 INFO Memo

+---------+
|total_sum|
+---------+
|       15|
+---------+



df: org.apache.spark.sql.package.DataFrame = [value: int]